In [1]:
"""
MNIST datasets demo for gcforest
Usage:
    define the model within scripts:
        python examples/demo_mnist.py
    get config from json file:
        python examples/demo_mnist.py --model examples/demo_mnist-gc.json
        python examples/demo_mnist.py --model examples/demo_mnist-ca.json
"""
import argparse
import numpy as np
import sys
from keras.datasets import mnist
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")

from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json

Using TensorFlow backend.
/Users/rajeshkumarsugumar/.virtualenvs/myvenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/rajeshkumarsugumar/.virtualenvs/myvenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/rajeshkumarsugumar/.virtualenvs/myvenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np

In [2]:
def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 100
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 10
    ca_config["estimators"] = []
    ca_config["estimators"].append(
            {"n_folds": 5, "type": "XGBClassifier", "n_estimators": 10, "max_depth": 5,
             "objective": "multi:softprob", "silent": True, "nthread": -1, "learning_rate": 0.1} )
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "ExtraTreesClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config

In [4]:
config = get_toy_config()
gc = GCForest(config)

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [10]:
print("The shape of the X_train", X_train.shape)
print("The shape of the y_train", y_train.shape)
print("The shape of the X_test", X_test.shape)
print("The shape of the y_test", y_test.shape)

The shape of the X_train (60000, 28, 28)
The shape of the y_train (60000,)
The shape of the X_test (10000, 28, 28)
The shape of the y_test (10000,)


In [11]:
X_train = X_train[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]

In [ ]:
X_train_enc = gc.fit_transform(X_train, y_train)

[ 2019-09-10 21:32:32,308][cascade_classifier.fit_transform] X_groups_train.shape=[(60000, 1, 28, 28)],y_train.shape=(60000,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2019-09-10 21:32:32,348][cascade_classifier.fit_transform] group_dims=[784]
[ 2019-09-10 21:32:32,349][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-09-10 21:32:32,350][cascade_classifier.fit_transform] group_ends=[784]
[ 2019-09-10 21:32:32,351][cascade_classifier.fit_transform] X_train.shape=(60000, 784),X_test.shape=(0, 784)
[ 2019-09-10 21:32:32,458][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(60000, 784), X_cur_test.shape=(0, 784)
[ 2019-09-10 21:35:22,553][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_0.predict)=90.08%
[ 2019-09-10 21:38:15,352][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_1.predict)=89.74%
[ 2019-09-10 21:41:04,886][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator

In [ ]:
y_pred = gc.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of GcForest = {:.2f} %".format(acc * 100))